<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto estará utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# Montagem do Drive (utilização do Google Colab)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# **Instalação de Bibliotecas - Setup**

In [1]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import matplotlib.pyplot as plt
import random

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import PlaintextCorpusReader
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import tokenize
from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## Loading Data
Fiz algumas alterações em algumas linhas do dataset, devido o encontro de alguns erros. Além disso, fiz a leitura em arquivo XLSX. pois estava dando erro na leitura do arquivo em csv. 

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Módulo 6/projeto/base_nova_csv.csv')
df

,id,"""dataPublicada""","""autor""","""texto""","""sentimento""","""tipoInteracao""","""anomalia""","""probabilidadeAnomalia""","""linkPost""","""processado""","""contemHyperlink"""
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0
...,...,...,...,...,...,...,...,...,...,...,...
12350,3044013,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0,0
12351,3044212,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0,0
12352,3044213,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0
12353,3044412,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0,0


# **Exploratória** dos dados

In [4]:
df['"probabilidadeAnomalia"'].value_counts() # confirmando se há algum erro no dataset, pois havia erros nessa coluna em específico
# Entender qual a função dessa coluna!

15    994
37    360
44    305
34    305
40    298
     ... 
7       5
82      4
85      4
86      2
87      1
Name: "probabilidadeAnomalia", Length: 80, dtype: int64

In [5]:
df['"processado"'].value_counts() # Entender qual a função dessa coluna!

0    12355
Name: "processado", dtype: int64

In [6]:
df['"contemHyperlink"'].value_counts() # Entender qual a função dessa coluna!

0    12288
1       67
Name: "contemHyperlink", dtype: int64

In [7]:
df['"anomalia"'].value_counts() # Entender qual a função dessa coluna!

0    9608
1    2747
Name: "anomalia", dtype: int64

## Tratamento

In [8]:
# renomeando as colunas para retirar os aspas duplos
df = df.rename(columns={'"anomalia"' : 'anomalia', '"dataPublicada"' : 'dataPublicada', '"autor"' : 'autor', '"texto"' : 'texto', '"sentimento"' : 'sentimento', '"tipoInteracao"' : 'tipoInteracao', '"probabilidadeAnomalia"' : 'probabilidadeAnomalia', '"linkPost"' : 'linkPost', '"processado"' : 'processado',  '"contemHyperlink"' : 'contemHyperlink' })
df

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0
...,...,...,...,...,...,...,...,...,...,...,...
12350,3044013,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0,0
12351,3044212,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0,0
12352,3044213,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0,0
12353,3044412,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0,0


In [9]:
df['tipoInteracao'].value_counts()

marcação      5999
comentário    5389
resposta       967
Name: tipoInteracao, dtype: int64

In [10]:
tipo_interacao_counts = df['tipoInteracao'].value_counts()

fig = px.bar(tipo_interacao_counts, x=tipo_interacao_counts.index, y='tipoInteracao',
             labels={'tipoInteracao': 'Contagem', 'x': 'Tipo de Interação'},
             title='Distribuição dos tipos de Interações')
fig.show()

In [11]:
df['sentimento'].value_counts()

NEUTRAL     5344
POSITIVE    4487
NEGATIVE    2524
Name: sentimento, dtype: int64

In [12]:
# LEMBRAR: tentar distribuir os REELS entre os que estão dentro e fora do FEED
tipo_midia_counts = df['sentimento'].value_counts()

fig = px.bar(tipo_midia_counts, x=tipo_midia_counts.index, y='sentimento',
             labels={'sentimento': 'Contagem', 'x': 'Sentimentos'},
             title='Distribuição dos Sentimentos dos comentários do Instagram, em diferentes posts')
fig.show()

## Retirando colunas

In [13]:
df = df.drop(['processado', 'id', 'dataPublicada'], axis=1)
df

,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,contemHyperlink
0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0
3,lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0
4,vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0
...,...,...,...,...,...,...,...,...
12350,onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0
12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0
12352,jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0
12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0


## Filtrando para somente autores diferentes do BTG

In [ ]:
# retirando toda e qualquer informação provinda do btgpactual para fins de análise somente dos usuários
df = df.loc[df['autor'] != 'btgpactual']
df

,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,contemHyperlink
0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0
3,lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0
4,vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0
...,...,...,...,...,...,...,...,...
12350,onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0
12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0
12352,jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0
12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0


## Descrevendo as features:
- id: provável id que foi salvo aquela informação
- autor: proprietário da mensagem
- texto: conteúdo da mensagem
- sentimento: classificação sentimental daquele comentário
  - **positivo** - 4487
  - **neutro** - 5344
  - **negativo** - 2524
- tipoInteracao: modo que foi enviada a mensagem, um simples comentário ou uma resposta a um comentário, sendo os seguintes tipos e suas respectivas quantias:
  - **comentário** - 5389
  - **resposta** - 967
  - **marcação** - 5999

- dataPublicada: data de publicação do post
- anomalia: fraudes
- probabilidadeAnomalia: probabilidade de fraudes, feita por outra IA
- linkPost: link dos posts
- contemHyperlink: links das fraudes

## Retirando mais colunas

In [14]:
df

,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,contemHyperlink
0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0
3,lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0
4,vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0
...,...,...,...,...,...,...,...,...
12350,onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,1,54,https://www.instagram.com/reel/CllsqI7goSC/,0
12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,0,39,https://www.instagram.com/p/Cllw0CmNGkX/,0
12352,jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,1,52,https://www.instagram.com/p/CllkSBOuKlJ/#17960...,0
12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,0,43,https://www.instagram.com/reel/ClluX-cAoaf/,0


In [15]:
df = df.drop(['anomalia', 'probabilidadeAnomalia', 'contemHyperlink', 'linkPost'], axis=1)
df

,autor,texto,sentimento,tipoInteracao
0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação
1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação
2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação
3,lmviapiana,Minuto touro de ouro,POSITIVE,comentário
4,vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta
...,...,...,...,...
12350,onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação
12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação
12352,jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário
12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação


# Bag of Words (BoW)

- Coluna de conteúdo que iremos tratar e fazer a normalização textual

In [27]:
df # Os seguintes textos não possuem o 'btgpactual' como autor. uma amostra de 1000 frases

4787     @souericareis Mulher linda, tu és uma linda fl...
6349     O BISCHOFF GROUP está entre os destaques do Ra...
8610     Está esperando o que para operar com a platafo...
7167                     @gabcintra7 entao nao vai ninguem
11242    Reduziram limite aff,, vou pagar o que ainda d...
                               ...                        
2314     Jardim Europa em alta 🌆  4 suítes | 4 vagas | ...
2634     Encerrei minha conta e fui pro @btgpactual, lá...
3335     @giovanni.infra quando você vai fazer a queixa...
4885     Oi! O link on-line que recebi no e-mail não es...
4100     Boa tarde, gostaria de saber onde estão os luc...
Name: texto, Length: 5000, dtype: object

## Definição das funções

Tokenização + Retirada de Stopwords + Remoção de caracteres especiais

In [17]:
# definindo a função teste que recebe como parâmetro uma lista de frases e retorna uma lista de palavras sem as stopwords, caracteres especiais e tokenizada
def tokenize(frases):
  # definindo a lista que será retornada
  lista = []

  # importando o módulo nltk para a remoção das stopwords
  stopwords = nltk.corpus.stopwords.words('portuguese')

  # loop para percorrer a lista de frases
  for frase in frases:
    # removendo os caracteres especiais
    frasezinha = re.sub((r'[^\w\s]'), ' ', frase)

    # tokenizando a frase e alterando para minúsculo
    termos = nltk.word_tokenize(frasezinha.lower())

    # loop para percorrer a lista de termos
    for palavra in termos:

      # verificando se a palavra não está na lista de stopwords, se estiver não adiciona na lista  
      if palavra not in stopwords:

        # adicionando a palavra na lista
        lista.append(palavra)
        
  # retornando a lista
  return lista

In [19]:
# Função de geração de BOW
def gerando_bow(frases):
  vocab = tokenize(frases) # Função que gera o vocabulário a partir das frases já tratadas
  print(f"Lista de palavras das frases: \n{vocab} \n") # Imprime o vocabulário
  
  for frase in frases: # Loop para gerar o BOW

    bag_vetores = np.zeros(len(vocab)) # Cria um vetor de zeros com o tamanho do vocabulário

    palavras = nltk.word_tokenize(frase.lower()) # Tokeniza as palavras da frase

    for w in palavras: # Loop para contar as palavras

      for i, palavra in enumerate(vocab): # Loop para percorrer o vocabulário

        if palavra == w: # Se a palavra for igual a palavra do vocabulário

          bag_vetores[i] += 1 # Adiciona 1 ao vetor
          
    print(f"{frase} \n{bag_vetores}\n") # Imprime a frase e o vetor

## Teste das funções

In [28]:
df = df.sample(n=1000) # pegando uma quantia aleatória para facilitar na análise

In [29]:
df

9239    Obrigado a quem nos dá as informações sobre @j...
2918    Esse é meu 6o ano no worldeconomicforum e semp...
1311    Meu Deus agnt faz um teste ai na onde vcs mora...
9527    Muitas pessoas acreditam que investir é um pro...
1381    Tô a 7 Meses com mesmo limite do black e nunca...
                              ...                        
9850    Dúvidas sobre investimentos? Agende sua visita...
9216    Efeito eleições quero ver em janeiro como vai ...
8975    #Repost btgpactual with use.repost ・・・ Como ex...
9869    ANOTE NA AGENDA!  Dia 16 de outubro, às 15h, a...
1184    Obrigado a você amigo e cliente por toda confi...
Name: texto, Length: 1000, dtype: object

In [30]:
gerando_bow(df)

Lista de palavras das frases: 
['obrigado', 'dá', 'informações', 'sobre', 'jill', 'diaz', 'trade', 'incrível', 'assim', 'comentarista', 'afirmou', 'nesta', 'página', 'negociei', 'recebo', 'lucro', 'obrigado', 'pessoal', '6o', 'ano', 'worldeconomicforum', 'sempre', 'surpreendendo', 'qualidade', 'discussões', 'possibilidade', 'conectar', 'tantas', 'lideranças', 'time', 'família', 'faz', 'tudo', 'acontecer', 'comigo', 'vittorio', 'perona', 'senior', 'partner', 'ibd', 'andré', 'portilho', 'partner', 'head', 'digital', 'assets', 'huw', 'jenkins', 'vice', 'chairman', 'btgpactual', '32', 'reuniões', 'eventos', 'concluindo', 'sucesso', 'negócios', 'deus', 'agnt', 'faz', 'teste', 'ai', 'onde', 'vcs', 'mora', 'vai', 'algum', 'vc', 'terreno', 'abandonado', 'q', 'esteva', 'mato', 'igual', 'área', 'desmatada', 'joga', 'várias', 'semente', 'árvores', 'pra', 'ver', 'vai', 'nascer', 'alguma', 'marketing', 'muitas', 'pessoas', 'acreditam', 'investir', 'processo', 'burocrático', 'inacessível', 'apenas',